In [1]:
import pandas as pd
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob 

import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
from random import sample
import sklearn
import re
import string
import warnings
from bs4 import BeautifulSoup
import gensim
from gensim.models.phrases import Phrases
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from analysis_func.text_preproc import preproc_text
from analysis_func.old_text_preproc import preprocess_e2e, preprocess_pre_tokenizing
from analysis_func.topic_preproc import prepare_corp
from analysis_func.topic_model import topicmodel
from analysis_func.saveload_topicmodels import save_modelanddata, load_modelanddata
import datetime
import os
import time
import sys


from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
tuples = pd.read_csv("../lite_data/climate_tuples.csv", header=0)

In [3]:
tuples.head()

,Unnamed: 0,Unnamed: 0.1,author,created_utc,score,subreddit,type,g_subs,rank,bin
0,0,0,chabanais,1374889166,0.0,Conservative,comment,Conservative,87375.0,8.0
1,3,3,Patriot4evr,1566312474,6.0,Justfuckmyshitup,comment,Justfuckmyshitup,2518.0,5.0
2,9,9,Fake_William_Shatner,1533736128,1.0,worldnews,comment,worldnews,20017.0,6.0
3,10,10,hybrid_remix,1560872065,1.0,heroesofthestorm,comment,heroesofthestorm,991.0,3.0
4,11,11,kirkisartist,1531524275,0.0,omise_go,comment,omise_go,2953.0,2.0


In [4]:
vaxAuthors = list(set(tuples['author'].tolist()))

In [5]:
dlist = []
fnames = glob.glob("/data/shruti/conspirecovery/reddit/initial/content/authorContent*.gz")
for f in fnames:
    print(f)
    temp_F = pd.read_csv(f, header=0)
    temp_F = temp_F.loc[temp_F['author'].isin(vaxAuthors)]
    dlist.append(temp_F)
    
    

/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000000.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000001.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000002.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000003.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000004.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000005.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000006.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000007.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000008.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000009.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000010.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000011.gz
/data/shruti/conspirecovery/reddit/initi

In [6]:
data = pd.concat(dlist)
data.head()

,author,subreddit,created_utc,score,id,body,author_flair_text,type
3,jeepdave,todayilearned,1565252595,1.0,ewav3lm,Doesn't make the courts right.,NaN,comment
4,kriegson,progun,1539875448,5.0,e803e5h,"They always try to deflect, dismiss and de-per...",NaN,comment
6,flowirin,pics,1485329315,1.0,dcvnbrj,"yep, waaay less here",NaN,comment
7,nakedapedude,newzealand,1483421680,6.0,dbxn157,"Swap his vegemite for marmite, that's pretty m...",NaN,comment
8,ranman1124,MapPorn,1468941216,2.0,d5i7vpc,No.,NaN,comment


In [7]:
antivax = ["vaccineskeptics","VaccinesCause","HealthConspiracy", "VaxTalk", "VaccineIngredients"]
climateskeptic = ["climateskeptics","greenskeptics","climatechangebullshit","climatehoax","Globalwarmingisfake","climateconspiratard"]
chemprints = ["CHEMPRINTS","Chemtrail","chemtrails","chemtrailpilots"]
flatearth = ["notaglobe","theworldisflat","flatearth", "Flat_Earth", "Flatearther"]

In [8]:
climate_data  = data.loc[data['subreddit'].isin(climateskeptic)]
climate_data.head()

,author,subreddit,created_utc,score,id,body,author_flair_text,type
316,ThrombBone,climateskeptics,1545115904,1.0,ec14vna,Your models are not proof and your revisionist...,NaN,comment
333,thesnarkisaboojum,climateskeptics,1365866358,4.0,c9eewk3,Everything was pretty happy during the medieva...,NaN,comment
482,pr-mth-s,climateskeptics,1474383908,10.0,53ns5l,Most of Ben &amp; Jerry's 300K twitter followe...,NaN,submission
510,bugsbunny4pres,climateskeptics,1541689831,10.0,9vas5k,Climate Genocide Coming,NaN,submission
662,barttali,climateskeptics,1501193648,4.0,dktddwn,Did you notice that NASA said **AVERAGE SEPTEM...,NaN,comment


In [9]:
climate_data['subreddit'].unique()

array(['climateskeptics', 'Globalwarmingisfake', 'climateconspiratard'],
      dtype=object)

In [9]:
len(climate_data)

304943

In [10]:
climate_data = climate_data.dropna(subset=['body'])
len(climate_data)

304941

In [11]:
climate_data.head()

,author,subreddit,created_utc,score,id,body,author_flair_text,type
316,ThrombBone,climateskeptics,1545115904,1.0,ec14vna,Your models are not proof and your revisionist...,NaN,comment
333,thesnarkisaboojum,climateskeptics,1365866358,4.0,c9eewk3,Everything was pretty happy during the medieva...,NaN,comment
482,pr-mth-s,climateskeptics,1474383908,10.0,53ns5l,Most of Ben &amp; Jerry's 300K twitter followe...,NaN,submission
510,bugsbunny4pres,climateskeptics,1541689831,10.0,9vas5k,Climate Genocide Coming,NaN,submission
662,barttali,climateskeptics,1501193648,4.0,dktddwn,Did you notice that NASA said **AVERAGE SEPTEM...,NaN,comment


In [12]:
climate_data['clean_text'] = climate_data['body'].parallel_apply(lambda x: preprocess_e2e(x))
climate_data.head()

,author,subreddit,created_utc,score,id,body,author_flair_text,type,clean_text
316,ThrombBone,climateskeptics,1545115904,1.0,ec14vna,Your models are not proof and your revisionist...,NaN,comment,your models are not proof and your revisionist...
333,thesnarkisaboojum,climateskeptics,1365866358,4.0,c9eewk3,Everything was pretty happy during the medieva...,NaN,comment,everything was pretty happy during the medieva...
482,pr-mth-s,climateskeptics,1474383908,10.0,53ns5l,Most of Ben &amp; Jerry's 300K twitter followe...,NaN,submission,most of ben jerry s 300k twitter followers w...
510,bugsbunny4pres,climateskeptics,1541689831,10.0,9vas5k,Climate Genocide Coming,NaN,submission,climate genocide coming
662,barttali,climateskeptics,1501193648,4.0,dktddwn,Did you notice that NASA said **AVERAGE SEPTEM...,NaN,comment,did you notice that nasa said average septembe...


In [13]:
climate_data.sample(5)

,author,subreddit,created_utc,score,id,body,author_flair_text,type,clean_text
1896154,Will_Power,climateskeptics,1397246524,1.0,cgq207w,Unbecoming of a moderator? How about this. Y...,NaN,comment,unbecoming of a moderator how about this you...
1888780,bugsbunny4pres,climateskeptics,1539716139,2.0,e7vxe91,Gates makes money using capitalism then partne...,NaN,comment,gates makes money using capitalism then partne...
1963299,Phatpharm269,climateskeptics,1558709163,1.0,eonciut,The constantly moving goalposts of AGW shows t...,NaN,comment,the constantly moving goalposts of agw shows t...
342818,JRugman,climateskeptics,1279051998,1.0,c0u6lhe,Naomi Oreskes has a very good presentation bas...,NaN,comment,naomi oreskes has a very good presentation bas...
156502,deck_hand,climateskeptics,1395163625,2.0,cg5qw5b,See my other comment. It seems relevant.,NaN,comment,see my other comment it seems relevant


In [14]:
swords = []
with open("../lite_data/stopwords.txt", "r") as rfile:
    for line in rfile:
        swords.append(line.strip("\n"))

print(swords)

['a', 'able', 'about', 'above', 'abst', 'accordance', 'according', 'accordingly', 'across', 'act', 'actually', 'added', 'adj', 'affected', 'affecting', 'affects', 'after', 'afterwards', 'again', 'against', 'ah', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'announce', 'another', 'any', 'anybody', 'anyhow', 'anymore', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apparently', 'approximately', 'are', 'aren', 'arent', 'arise', 'around', 'as', 'aside', 'ask', 'asking', 'at', 'auth', 'available', 'away', 'awfully', 'b', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'begin', 'beginning', 'beginnings', 'begins', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'between', 'beyond', 'biol', 'both', 'brief', 'briefly', 'but', 'by', 'c', 'ca', 'came', 'can', 'cannot', "can't", 'cause', 'causes', 'certain', 'certainly', 'co', 'com', 'come', 'comes', 'cont

In [15]:
def remove_stops(text):
    new_text = " ".join([t for t in text.split(" ") if t not in swords])
    if len(new_text) > 5:
        return new_text
    
climate_data['cleaner_text'] = climate_data['clean_text'].parallel_apply(lambda x: remove_stops(x))

In [16]:
climate_data.sample(5)

,author,subreddit,created_utc,score,id,body,author_flair_text,type,clean_text,cleaner_text
1437330,ENGLISHSCONESYAKNOW,climateskeptics,1569720605,1.0,f1rv7hf,Correct but if the US stopped all emmesion tod...,NaN,comment,correct but if the us stopped all emmesion tod...,correct stopped emmesion today set major europ...
1140002,Will_Power,climateskeptics,1288154137,0.0,c13k2es,Why should I? They had already been downvoted...,NaN,comment,why should i they had already been downvoted ...,downvoted folks good commenting articles
1299175,Digglord,climateskeptics,1555901675,6.0,elgxr09,Do you think humans contribute more to the cha...,NaN,comment,do you think humans contribute more to the cha...,humans contribute changing climate natural for...
976585,PerniciousParagon,climateskeptics,1517540281,1.0,dtlvf8n,"As much of an anti-alarmist as I am, this is a...",NaN,comment,as much of an anti alarmist as i am this is a...,anti alarmist local phenomena evidence global...
245198,bean-a,climateskeptics,1501536581,7.0,dkzfyez,Gore will dress up as an ancient Mayan and do ...,NaN,comment,gore will dress up as an ancient mayan and do ...,gore dress ancient mayan big circle dance film


In [17]:
climate_data = climate_data.dropna(subset=['cleaner_text'])

In [18]:
len(climate_data)

298014

In [19]:
print("prepping corpus for topicmodel")
# prepare data for topic modeling
corp = climate_data['cleaner_text'].tolist()
corp_tokens = [c.split() for c in corp]
phrases = Phrases(corp_tokens, min_count=5, threshold=0.1)
tokes = [phrases[t] for t in corp_tokens]

prepping corpus for topicmodel


In [20]:
climate_data.to_csv("/data/shruti/conspirecovery/reddit/initial/smallConsp/climate_data.csv")

In [21]:
climate_data.head()

,author,subreddit,created_utc,score,id,body,author_flair_text,type,clean_text,cleaner_text
316,ThrombBone,climateskeptics,1545115904,1.0,ec14vna,Your models are not proof and your revisionist...,NaN,comment,your models are not proof and your revisionist...,models proof revisionist history ethical
333,thesnarkisaboojum,climateskeptics,1365866358,4.0,c9eewk3,Everything was pretty happy during the medieva...,NaN,comment,everything was pretty happy during the medieva...,pretty happy medieval warm period edit len...
482,pr-mth-s,climateskeptics,1474383908,10.0,53ns5l,Most of Ben &amp; Jerry's 300K twitter followe...,NaN,submission,most of ben jerry s 300k twitter followers w...,ben jerry 300k twitter followers flavors lo...
510,bugsbunny4pres,climateskeptics,1541689831,10.0,9vas5k,Climate Genocide Coming,NaN,submission,climate genocide coming,climate genocide coming
662,barttali,climateskeptics,1501193648,4.0,dktddwn,Did you notice that NASA said **AVERAGE SEPTEM...,NaN,comment,did you notice that nasa said average septembe...,notice nasa average september extent word s...


In [27]:
climate_data.sample(10)

,author,subreddit,created_utc,score,id,body,author_flair_text,type,clean_text,cleaner_text
1171179,bugsbunny4pres,climateskeptics,1538159087,8.0,9jpgjc,The Green Jobs Fallacy,NaN,submission,the green jobs fallacy,green jobs fallacy
1996417,nopenictesla,climateskeptics,1392046820,-2.0,cfbpzyq,In this case also the blog post is deceptive a...,NaN,comment,in this case also the blog post is deceptive a...,case blog post deceptive intentionally mislead...
180045,matmyob,climateskeptics,1574159114,-1.0,f81aruk,And currently in Australia today: extreme heat...,NaN,comment,and currently in australia today extreme heat...,australia today extreme heat 48°c catastrop...
1126094,logicalprogressive,climateskeptics,1407015409,3.0,cjf65e8,Alarmist global warming is becoming increasing...,NaN,comment,alarmist global warming is becoming increasing...,alarmist global warming increasingly irrelevan...
295870,Vedoom123,climateskeptics,1571880624,4.0,f4yng7g,It's not like we're trees. We have legs and we...,NaN,comment,it s not like we re trees we have legs and we...,trees legs cars sea level rise slow worry a...
1944025,xray606,climateskeptics,1491285502,10.0,dft0a4v,"It would to rational people, but...",NaN,comment,it would to rational people but,rational people
601076,YehNahYer,climateskeptics,1573069096,1.0,f6q1o5b,I ride a cow that farts methane.,NaN,comment,i ride a cow that farts methane,ride cow farts methane
1255383,Tunderbar1,climateskeptics,1482898265,1.0,dbpdsus,They are wrong in the context that climate ala...,NaN,comment,they are wrong in the context that climate ala...,wrong context climate alarmists 85 years for...
1031658,UncleEggma,climateskeptics,1412976206,0.0,cl61jv7,"Once again, you ignore everything I've said an...",NaN,comment,once again you ignore everything i ve said an...,ignore ve skeptic child
304354,AlyssaMoore,climateskeptics,1394997422,3.0,cg44tn9,Another thing that needs to be added to the li...,NaN,comment,another thing that needs to be added to the li...,thing list


In [28]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from analysis_func.text_preproc import preproc_text
from analysis_func.topic_preproc import prepare_corp
from analysis_func.topic_model import topicmodel
from analysis_func.saveload_topicmodels import save_modelanddata, load_modelanddata

In [ ]:
%%time
id2word, newcorpus = prepare_corp(tokes)
model = topicmodel(10, newcorpus, id2word)

In [ ]:
from pprint import pprint
pprint(model.print_topics())
doc_lda = model[newcorpus]